In [78]:
import os
import json
import pandas as pd

results_dir = "../data/results/longmemeval_short_RAG_openai_gpt-5-mini_openai_gpt-5-mini"
json_files = [f for f in os.listdir(results_dir) if f.endswith(".json")]

results = []
for filename in json_files:
    filepath = os.path.join(results_dir, filename)
    with open(filepath, "r") as f:
        data = json.load(f)
        results.append(data)

df = pd.DataFrame(results)
df

,question_id,question,predicted_answer,answer,answer_is_correct
0,d24813b1,I'm thinking of inviting my colleagues over fo...,"Nice — here are practical, crowd‑friendly baki...",The user would prefer baking suggestions that ...,True
1,15745da0,How long have I been collecting vintage cameras?,Three months.,three months,True
2,gpt4_2f8be40d,How many weddings have I attended in this year?,At least three. The evidence mentions: your co...,I attended three weddings. The couples were Ra...,True
3,gpt4_483dd43c,"Which show did I start watching first, 'The Cr...",You started Game of Thrones first — you said y...,'Game of Thrones',True
4,c7dc5443,What is my current record in the recreational ...,Your current record is 5-2.,5-2,True
...,...,...,...,...,...
489,e5ba910e_abs,What is the total cost of my recently purchase...,"From the evidence, your Sony WH-1000XM4 headph...",The information provided is not enough. You me...,True
490,d7c942c3,Is my mom using the same grocery list method a...,"Yes. You said, ""my mom is actually using the s...",Yes.,True
491,gpt4_45189cb4,What is the order of the sports events I watch...,There was one sports event mentioned that you ...,"First, I attended a NBA game at the Staples Ce...",False
492,dad224aa,What time do I wake up on Saturday mornings?,It varies. In the evidence you report waking a...,7:30 am,False


In [79]:
print(df['answer_is_correct'].mean())

0.7165991902834008


In [80]:
for _, row in df.iterrows():
    answer = row['answer']
    predicted_answer = row['predicted_answer']
    correct = row['answer_is_correct']
    print(f"Question: {row['question']}")
    print(f"Answer: {answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Correct: {correct}")
    print("\n")


Question: I'm thinking of inviting my colleagues over for a small gathering. Any tips on what to bake?
Answer: The user would prefer baking suggestions that take into account their previous success with the lemon poppyseed cake, such as variations of that recipe or other desserts that share similar qualities. They might prefer suggestions that balance impressiveness with manageability, considering their previous experience. The user may not prefer overly complex or unfamiliar recipes, or suggestions that do not build upon their existing baking experience.
Predicted Answer: Nice — here are practical, crowd‑friendly baking ideas + tips so you can pick what fits your time, equipment, and your colleagues’ tastes.

What to bake (mix sweet + savory, and one “showstopper”)
- Signature cake (showstopper): a lemon cake is perfect — consider lemon blueberry, lemon curd layer, or lemon‑lavender pound cake for something a bit special (light, bright, and easy to slice). If you already have a great 

In [81]:
with open("../data/longmemeval/longmemeval_s_cleaned.json", "r", encoding="utf-8") as f:
    longmemeval_df = pd.DataFrame(json.load(f)).reset_index(drop=True)

In [82]:
df.merge(longmemeval_df[["question_id", "question_type"]], on="question_id", how="inner").groupby(
    "question_type"
).agg(
    answer_is_correct_mean=("answer_is_correct", "mean"), count=("answer_is_correct", "count")
).sort_values(
    by="answer_is_correct_mean", ascending=False
).reset_index()

,question_type,answer_is_correct_mean,count
0,single-session-assistant,0.982143,56
1,single-session-user,0.955882,68
2,single-session-preference,0.766667,30
3,multi-session,0.748092,131
4,knowledge-update,0.675325,77
5,temporal-reasoning,0.462121,132


In [83]:
longmemeval_df['question_type'].value_counts()

question_type
multi-session                133
temporal-reasoning           133
knowledge-update              78
single-session-user           70
single-session-assistant      56
single-session-preference     30
Name: count, dtype: int64